# Simulation of fast neutron flux

Fast neutron flux is of particular interest to fusion neutronics analysis as fast neutrons are able to undergo nuclear reactions with nuclei (threshold reactions) that slower neutrons can not.

The fast neutron flux is used to estimate super conducting magnet lifetime amount other things

First import OpenMC and configure the nuclear data path

In [ ]:
import openmc

# Setting the cross section path to the correct location in the docker image.
# If you are running this outside the docker image you will have to change this path to your local cross section path.
openmc.config['cross_sections'] = '/nuclear_data/cross_sections.xml'

In [ ]:
import openmc

dag_univ = openmc.DAGMCUniverse(filename="dagmc.h5m").bounded_universe(padding_distance=1)

umesh = openmc.UnstructuredMesh("dagmc.vtk", library="moab")
mesh_filter = openmc.MeshFilter(umesh)
energy_filter = openmc.EnergyFilter([10.0e6, 20.0e6])  # filtering neutrons between 10MeV and 20MeV, this is just an example and your definition of fast neutron may vary

tally = openmc.Tally(name="unstructured_mesh_tally")
tally.filters = [mesh_filter, energy_filter]
tally.scores = ["flux"]
my_tallies = openmc.Tallies([tally])

mat1 = openmc.Material(name='mat1')
mat1.add_nuclide("H1", 1, percent_type="ao")
mat1.set_density("g/cm3", 0.001)
my_materials = openmc.Materials([mat1])

my_geometry = openmc.Geometry(root=dag_univ)

my_settings = openmc.Settings()
my_settings.batches = 10
my_settings.particles = 5000
my_settings.run_mode = "fixed source"

# Create a DT point source
my_source = openmc.IndependentSource()
my_source.space = openmc.stats.Point((0, 0, 0.5))  # this is the center of the geometry
my_source.angle = openmc.stats.Isotropic()
my_source.energy = openmc.stats.Discrete([14e6], [1])
my_settings.source = my_source

model = openmc.model.Model(my_geometry, my_materials, my_settings, my_tallies)
sp_filename = model.run()

sp = openmc.StatePoint(sp_filename)

tally_result = sp.get_tally(name="unstructured_mesh_tally")

# It is nessecary to get the mesh from statepoint from as it contains more information than the mesh loaded from the vtk file. 

umesh_from_sp = tally_result.find_filter(openmc.MeshFilter).mesh

flux_mean = tally_result.get_values(scores=["flux"], value="mean").reshape(umesh_from_sp.dimension)
umesh_from_sp.write_data_to_vtk(filename="tally.vtk", datasets={"mean": flux_mean})